<a href="https://colab.research.google.com/github/daxzy-ai/SD-Anime-versions-webui/blob/main/N_AILeaks_API_Backend_EN_ID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on 4chan NovelAILeaks (naifu)[src](https://boards.4channel.org/g/thread/89095460#p89097704)

updated on 07/11/2022 16.49 GMT +7

another update on 13/12/2022 19.33 GMT +7, added the backround music to keep the tabs open for mobile device (thanks KoboldAI)

In [ ]:
#@title ### 0. Check GPU working status

!nvidia-smi

In [ ]:
#@title ### 0.5. FOR MOBILE USER (Untuk pengguna Android)

%%html
<b>Press play on the music player to keep the tab alive, then continue the next step (Uses only 13MB of data)</b><br/>
<br/>
<i>Untuk pengguna mobile, tekan play di musik di bawah, trus baru lanjut ke langkah berikutnya</i><br/>
<i>Fungsinya agar bisa make Colabnya lebih lama (make kuota 13MB)</i><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title ### 1. Download Novel AI API backend, model
#@markdown If the download speed is too slow try restart

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

In [ ]:
#@title ### 2. Install dependencies
#@markdown Wait patiently for the installation to complete<br/>
#@markdown *(Sabar aja tunggu ampe downloadnya kelar)*

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared
!/content/naifu/venv/bin/python -m pip install -qq pytorch_lightning==1.7.7

In [ ]:
#@title ### 3. Start the model
#@markdown Just visit the output mapping address (end with `trycloudflare.com` / `bore.pub`).
#@markdown - Please wait until the model is loaded (`Application startup complete` appears) before accessing
#@markdown - The service provided by cloudflare occasionally has a request timeout, which can be replaced by bore tunnel

#@markdown *Klik link yang ada `trycloudflare.com`, atau copas link yang `bore.pub`).*
#@markdown - *Tunggu ampe model kelar diproses (muncul `Application startup complete`) baru klik link*
#@markdown - *Kalo yg cloudflare gabisa, pakai bore.pub*

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

In [ ]:
#@title ### 4. (Optional) Run with `animefull-latest` model of 7G
#@markdown uses the 4G size animefull-final-pruned model by default. If you want to use the 7G animefull-latest model, run this

%cd /content/
!mkdir /content/naifu/models/animefull-latest
#!tar xf animefulfull-latest && rm animefull-lal-latest.tar -C /content/naifu/models/animetest.tar
#!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!tar xf animefull-latest.tar -C /content/naifu/models/animefull-latest && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969